In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable
import astropy.units as u
import sys
import pyneb as pn
from astropy.io import fits
import aplpy
from matplotlib import gridspec
from astropy.io import ascii 
import scipy
from scipy.optimize import curve_fit
from scipy.integrate import quad
import numpy.linalg as la

### Import the spectra data for NGC 4254

In [ ]:
hdu1 = fits.open('/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/NGC4254_VorSpectra.fits')

### Import PHANGS and refitted data

In [ ]:
infile = open("/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
galaxy = 'NGC4254'
data = hdul[hdul['gal_name'] == f'{galaxy}']

infile = open("/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/Nebulae_catalogue_v2_refitNII_refitSIII.fits",'rb')
hdul = Table.read(infile)
galaxy = 'NGC4254'
siiidata = hdul[hdul['GAL_NAME'] == f'{galaxy}']

infile = open("/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/Nebulae_catalogue_v2_refitNII_refitTe.fits",'rb')
hdul = Table.read(infile)
galaxy = 'NGC4254'
niidata = hdul[hdul['GAL_NAME'] == f'{galaxy}']

### Define variables for the spectra and wavelength data

In [ ]:
lam = np.exp(hdu1[2].data['LOGLAM'])
log_spec = hdu1[1].data['SPEC']

### Find the Spectral Resolution ($\sigma$) using the given range from Emsellem et al. 2020 and the doppler shift equation

In [ ]:
deswave = 7319
dopv = 126.47 - 0.00978*deswave
delwav = (deswave * dopv * 10**13) / (3 * 10**8 * 10**10)

### Pick a region and set the boundaries for fitting

In [ ]:
regnum = 1557     #173 seems to be the brightest region in the catalog. Other good regions: 1557
lowerbound = 7280
wave0 = 7320*(data[regnum]['HA6562_VEL']+2388)/(299792) + 7320
upperbound = wave0 + 5
wavrange = lam[np.where(lam > lowerbound)[0][0]:np.where(lam > upperbound)[0][0]]
fluxrange = log_spec[regnum][np.where(lam > lowerbound)[0][0]:np.where(lam > upperbound)[0][0]]
wave0

### The code below defines a guassian distribution and fits parameters to the 'regnum' NGC 4254 region for the OII 7319 line

In [ ]:
def gauss(x, C, a):    #a:amplitude, x0:average wavelength, sig:stdev, C:zero offeset
    return a * np.exp((-(x-wave0) ** 2)/ (2 * delwav ** 2)) + C

fluxave = np.mean(fluxrange)
p0 = np.array([fluxave, 500])  #1 / (2*np.sqrt(2*np.log(2)))

param, paramcov = curve_fit(gauss, wavrange, fluxrange, p0)
param

### The code below defines a function without the 'C' offset

In [ ]:
def gauss_noC(x, a):  #a:amplitude, x0:average wavelength, sig:stdev, C:zero offeset
    sig = 1.339137
    return a * np.exp((-(x-wave0) ** 2)/ (2 * sig ** 2))

### The code below uses scipy's integrate.quad() function to find the flux values from the gaussian fits

In [ ]:
ftest = quad(gauss_noC, wave0 - 10, wave0 + 10, args=(param[1]))[0]
noise1 = np.std(log_spec[regnum][np.where(lam > wave0 - 100)[0][0]:np.where(lam > wave0 - 20)[0][0]])
noise2 = np.std(log_spec[regnum][np.where(lam > wave0 + 40)[0][0]:np.where(lam > wave0 + 100)[0][0]])
noise = (noise1 + noise2)/2
signal = param[1]
ston = signal/noise
print(f'A flux value of {ftest} with a S/N of {ston}')
intrange = lam[np.where(lam > wave0 - 10 * delwav)[0][0]:np.where(lam > wave0 + 10 * delwav)[0][0]]

### The code below plots the spectra of the first region in NGC 4254

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 4)

plt.plot(lam, log_spec[regnum], label='Emission Spectrum')
plt.axvline(7330, color='r', label='7330 Angstrom')
plt.axvline(7319, color='purple', label='7320 Angstrom')
#plt.plot(wavrange, fluxrange, color='orange', linestyle='--', label='Gaussian Fit Range')
plt.plot(wavrange, gauss(wavrange, param[0], param[1]), color='green', label='Gaussian Fit')
plt.axvline(wave0, color='cyan', linestyle='--', label='Redshifted [O II]7320')
plt.fill_between(lam[np.where(lam > wave0 - 100)[0][0]:np.where(lam > wave0 - 20)[0][0]], param[0]*100, color='purple', alpha = 0.15, label='Noise')
plt.fill_between(lam[np.where(lam > wave0 + 40)[0][0]:np.where(lam > wave0 + 100)[0][0]], param[0]*100, color='purple', alpha = 0.15)

plt.xlim(wave0-120, wave0+120)
plt.ylim(fluxave - fluxave*0.75, fluxave + fluxave*0.75)
plt.legend(loc='best')
plt.xlabel('Wavelength [Angstrom]', fontsize=15)
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]', fontsize=15)

### The code below is a function for refitting the [OII]7319 line

In [ ]:
def refit7319(inwave, influx, snval, phdata, galvel):
    
    def gaussian(x, C, a):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2))

    signoise = np.zeros([len(influx)])
    delwave = 1.3391374247333334
    outflux = np.zeros(len(influx))
    wavelength = 7320
    for i in range(len(influx)):
        lowb = 7280
        w0 = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        highb = w0 + 5        #plus five to not include [O II]7330 peak
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, w0 - 10 * delwave, w0 + 10 * delwave, args=(param[1]))[0]
        signal = param[1]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 100)[0][0]:np.where(inwave > w0 - 20)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 + 40)[0][0]:np.where(inwave > w0 + 100)[0][0]])
        noise = (noise1 + noise2)/2 
        fluxpeak = np.max(influx[i][np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]])
        rangepeak = np.max(fluxes)
        signoise[i] = signal/noise
        
        if signal/noise > snval and flux > 0: #and fluxpeak == rangepeak: 
            outflux[i] = flux
        else: 
            outflux[i] = np.nan
    
    return outflux, signoise

In [ ]:
ref7319, snlist = refit7319(lam, log_spec, 3, data, 2388)
ref7319

## Plot the Refitted [O II]7319 lines with the refitted [N II]5754 lines

In [ ]:
plt.scatter(niidata['NII5754_FLUX_CORR'], niidata['NII5754_FLUX_CORR_REFIT'], s=1, c='blue', label='K.Kreckel - [NII]5754')
plt.scatter(ref7319, data['OII7319_FLUX'], s=10, c='red', label='Eric - [OII]7319')
plt.xlabel(r'Refitted Lines [$\frac{erg}{cm^{2}s}$]')
plt.ylabel(r'PHANGS-MUSE Nebular Catalog [$\frac{erg}{cm^{2}s}$]')
plt.legend(loc='upper left')
#plt.xlim(-10**3, 0.3 * 10**5)
#plt.ylim(-10**3, 10**4)
plt.grid()

In [ ]:
count = 0
for i in range(len(snlist)):
    if snlist[i] > 3:
        count += 1
print(count)

In [ ]:
count = 0
for i in range(len(ref7319)):
    if np.isnan(ref7319[i]) == False:
        count += 1
print(count)

# [O II]7330 Refitting

### Starting values for [OII]7330 line

In [ ]:
rnum = 1557
wave7330 = 7330*(data[regnum]['HA6562_VEL']+2388)/(299792) + 7330
lbound = wave7330 - 5
ubound = wave7330 + 100
wrange = lam[np.where(lam > lbound)[0][0]:np.where(lam > ubound)[0][0]]
frange = log_spec[rnum][np.where(lam > lbound)[0][0]:np.where(lam > ubound)[0][0]]
deswave = 7330
dopv = 126.47 - 0.00978*deswave
del7330 = (deswave * dopv * 10**13) / (3 * 10**8 * 10**10)
wave7330

### [OII]7330 Gaussian Fit

In [ ]:
def gauss7330(x, C, a):    #a:amplitude, x0:average wavelength, sig:stdev, C:zero offeset
    return a * np.exp((-(x-wave7330) ** 2)/ (2 * del7330 ** 2)) + C

def gauss_noC_7330(x, a):  #a:amplitude, x0:average wavelength, sig:stdev, C:zero offeset
    return a * np.exp((-(x-wave0) ** 2)/ (2 * del7330 ** 2))

### Fit [OII]7330 line

In [ ]:
fave = np.mean(frange)
p0 = np.array([fave, 500])

param, paramcov = curve_fit(gauss7330, wrange, frange, p0)
param

### Use fit parameters to find flux and S/N

In [ ]:
flux = quad(gauss_noC_7330, wave7330 - 10 * del7330, wave7330 + 10 * del7330, args=(param[1]))[0]
sig = param[1]
noise1 = np.std(log_spec[regnum][np.where(lam > wave7330 - 100)[0][0]:np.where(lam > wave7330 - 20)[0][0]])
noise2 = np.std(log_spec[regnum][np.where(lam > wave7330 + 40)[0][0]:np.where(lam > wave7330 + 100)[0][0]])
noise = (noise1 + noise2)/2
ston = sig/noise
print(f'A flux value of {flux} with a S/N of {ston}')

### Plot continuum and fit

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 4)

plt.plot(lam, log_spec[rnum], label='Emission Spectrum')
plt.axvline(7330, color='r', label='7330 Angstrom')
plt.axvline(7319, color='purple', label='7319 Angstrom')
#plt.plot(wrange, frange, color='orange', linestyle='--', label='Gaussian Fit Range')
plt.plot(wrange, gauss7330(wrange, param[0], param[1]), color='green', label='Gaussian Fit')
plt.axvline(wave7330, color='cyan', linestyle='--', label='Redshifted [O II]7330')
plt.fill_between(lam[np.where(lam > wave7330 - 100)[0][0]:np.where(lam > wave7330 - 20)[0][0]], param[0]*100, color='purple', alpha = 0.15, label='Noise')
plt.fill_between(lam[np.where(lam > wave7330 + 40)[0][0]:np.where(lam > wave7330 + 100)[0][0]], param[0]*100, color='purple', alpha = 0.15)

plt.xlim(7250, 7500)
plt.ylim(fave - fave*0.75, fave + fave*0.75)
plt.legend(loc='lower right')
plt.xlabel('Wavelength [Angstrom]', fontsize=15)
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]', fontsize=15)

### Below is a function to refit the [OII]7330 line 

In [ ]:
def refit7330(inwave, influx, snval, indata, galvel):
    outflux = np.zeros(len(influx))
    wave = 7330
    dopv = 126.47 - 0.00978*wave
    d7330 = (wave * dopv * 10**13) / (3 * 10**8 * 10**10)
    
    def gaussian(x, C, a):    #a:amplitude, wavelength:feature wavelength, d7330:spectral resolution, C:zero offeset
        return a * np.exp((-(x-wavelength) ** 2)/ (2 * d7330 ** 2)) + C
    
    def gaussian_noC(x, a): 
        return a * np.exp((-(x-wavelength) ** 2)/ (2 * d7330 ** 2))
    
    for i in range(len(influx)):
        wavelength = wave*(indata[i]['HA6562_VEL']+galvel)/(299792) + wave
        lowb = wavelength - 5
        highb = wavelength +100
        waves = inwave[np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]]
        fluxes = influx[i][np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, wavelength - 10 * d7330, wavelength + 10 * d7330, args=(param[1]))[0]
        sig = param[1]
        noise1 = np.std(influx[i][np.where(inwave > wavelength - 100)[0][0]:np.where(inwave > wavelength - 20)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > wavelength + 40)[0][0]:np.where(inwave > wavelength + 100)[0][0]])
        noise = (noise1 + noise2)/2
        fluxpeak = np.max(influx[i][np.where(inwave > lowb + 20)[0][0]:np.where(inwave > highb)[0][0]])
        rangepeak = np.max(fluxes)
        if sig/noise > snval and flux > 0 and fluxpeak == rangepeak:
            outflux[i] = flux
        else: 
            outflux[i] = np.nan
    
    return outflux

In [ ]:
ref7330 = refit7330(lam, log_spec, 3, data, 2388)
ref7330

In [ ]:
plt.scatter(niidata['NII5754_FLUX_CORR'], niidata['NII5754_FLUX_CORR_REFIT'], s=1, c='blue', alpha = 0.5, label='[NII]5754')
#plt.scatter(ref7319, data['OII7319_FLUX'], s=1, c='red', alpha =0.5, label='[OII]7319')
plt.scatter(ref7330, data['OII7330_FLUX'], s=100, c='green', alpha =0.5, label='[OII]7330')
plt.xlabel(r'Refitted Lines [$\frac{erg}{cm^{2}s}$]')
plt.ylabel(r'PHANGS Catalog [$\frac{erg}{cm^{2}s}$]')
plt.legend(loc='upper left')
plt.xlim(-10**3, 10**4)
plt.ylim(-10**3, 10**4)
plt.grid()

In [ ]:
count = 0
for i in range(len(ref7330)):
    if np.isnan(ref7330[i]) == False:
        count += 1
print(count)

In [ ]:
count = 0
for i in range(len(ref7319)):
    if np.isnan(ref7319[i]) == False:
        count += 1
print(count)

# Refitting the [NII]5755 line

In [ ]:
dopv = 126.47 - 0.00978*niiwav
delwave = (niiwav * dopv * 10**13) / (3 * 10**8 * 10**10)
delwave

In [ ]:
rnum = 1557
niiwav = 5755
redwave = niiwav*(data[rnum]['HA6562_VEL']+2388)/(299792) + niiwav
#redwave = niiwav*(2388)/(299792) + niiwav
low = redwave - 50
#high = lam[np.where(log_spec[rnum] == 0)[0][0]-1]
high = redwave + delwave*2.2
waverange = lam[np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]
fluxrange = log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]
fluxave = np.mean(fluxrange)
p0list = np.array([fluxave, 500])
redwave

In [ ]:
def gaussian(x, C, a):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-redwave) ** 2)/ (2 * delwave ** 2)) + C
    
def gaussian_noC(x, a):
    return a * np.exp((-(x-redwave) ** 2)/ (2 * delwave ** 2))

In [ ]:
param, paramcov = curve_fit(gaussian, waverange, fluxrange, p0list)
flux = quad(gaussian_noC, redwave - 10 * delwave, redwave + 10 * delwave, args=(param[1]))[0]
signal = param[1]
noise1 = np.std(log_spec[rnum][np.where(lam > redwave - 150)[0][0]:np.where(lam > redwave - 30)[0][0]])
noise2 = np.std(log_spec[rnum][np.where(lam > redwave + 180)[0][0]:np.where(lam > redwave + 250)[0][0]])
noise = (noise1+noise2)/2
print(f'For region {rnum} a [NII]5755 flux value of {flux} was found with a S/N of {signal/noise}')

In [ ]:
redwave = 5755*(data[rnum]['HA6562_VEL']+2388)/(299792) + 5755

fig = plt.figure()
fig.set_size_inches(10, 4)

plt.axvline(5755, color='green', linestyle='--', label='5755 Angstrom')
plt.axvline(redwave, color='cyan', linestyle='--', label='Redshifted [N II]5755')
plt.plot(waverange, gaussian(waverange, param[0], param[1]), color = 'black', linestyle='--', label='Gaussian Fit')
plt.fill_between(lam[np.where(lam > redwave - 150)[0][0]:np.where(lam > redwave - 30)[0][0]], param[0]*100, color='purple', alpha = 0.15, label='Noise')
plt.fill_between(lam[np.where(lam > redwave + 180)[0][0]:np.where(lam > redwave + 250)[0][0]], param[0]*100, color='purple', alpha = 0.15)
plt.plot(lam, log_spec[rnum], color = 'red', alpha = 0.5, label='Emission Spectrum')

low = redwave - 150
high = lam[np.where(log_spec[rnum] == 0)[0][0]]
plt.xlim(low - 25, high + 250)
peak = np.max(log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]])
plt.ylim(param[0] - 0.2*param[0], param[0] + 0.2*param[0])
plt.legend(loc='best')
plt.xlabel('Wavelength [Angstrom]', fontsize=15)
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]', fontsize=15)

In [ ]:
def refit5755(inwave, influx, phdata, wavelength, galvel, snval):
    
    def gaussian(x, C, a):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2))

    signoise = np.zeros([len(influx)])
    dopv = 126.47 - 0.00978*wavelength
    delwave = (wavelength * dopv * 10**13) / (3 * 10**8 * 10**10)
    outflux = np.zeros(len(influx))
    for i in range(len(influx)):
        w0 = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = w0 - 150
        highb = w0 + 2*delwave
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, w0 - 10 * delwave, w0 + 10 * delwave, args=(param[1]))[0]
        signal = param[1]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 150)[0][0]:np.where(inwave > w0 - 30)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 + 180)[0][0]:np.where(inwave > w0 + 250)[0][0]])
        noise = (noise1 + noise2)/2 
        fluxpeak = np.max(influx[i][np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]])
        rangepeak = np.max(fluxes)
        signoise[i] = signal/noise
        
        if signal/noise > snval and flux > 0: #and fluxpeak == rangepeak: 
            outflux[i] = flux
        else: 
            outflux[i] = np.nan
    
    return outflux, signoise

In [ ]:
ref5755, sn5755 = refit5755(lam, log_spec, data, 5755, 2388, 3)
ref5755

In [ ]:
count = 0
for i in range(len(ref5755)):
    if np.isnan(ref5755[i]) == False:
        count += 1
print(count)

In [ ]:
count = 0
for i in range(len(niidata['NII5754_FLUX_REFIT'])):
    if niidata[i]['NII5754_FLUX_REFIT'] > 3 * niidata[i]['NII5754_FLUX_REFIT_ERR']:
        count += 1
print(count)

In [ ]:
count = 0
for i in range(len(ref5755)):
    if sn5755[i] > 1.5:
        count += 1
print(count)

In [ ]:
plt.scatter(ref5755, niidata['NII5754_FLUX_REFIT'], s=10, c='blue', alpha = 0.5, label='[NII]5755')
plt.plot(np.linspace(-50, 10**5, 1000), np.linspace(-50, 10**5, 1000), c='black', label='1-to-1 line')
plt.xlabel(r'Eric - Refitted Lines NII [$\frac{erg}{cm^{2}s}$]')
plt.ylabel(r'K. Kreckel - Refitted Lines NII [$\frac{erg}{cm^{2}s}$]')
plt.legend(loc='upper left')
plt.xlim(0, 0.2 * 10**5)
plt.ylim(0, 0.2 * 10**5)
plt.grid()

In [ ]:
plt.scatter(ref5755, (data['NII5754_FLUX'] - ref5755)/ref5755, s=25, c='blue', label='Eric')
#plt.scatter(niidata['NII5754_FLUX_REFIT'], (data['NII5754_FLUX'] - niidata['NII5754_FLUX_REFIT'])/niidata['NII5754_FLUX_REFIT'], s=10, c='red', alpha = 0.5, label='K. Kreckel')
plt.xlabel(r'Refitted Lines NII [$\frac{erg}{cm^{2}s}$]')
plt.ylabel(r'Relative Difference between Refits and Nebular Catalog')
plt.legend(loc='upper right')
plt.xlim(-5, 0.2 * 10**5)
#plt.ylim(-100, 100)
plt.grid()

# Refitting the [SIII]6312 Line

In [ ]:
rnum = 93
siiiwav = 6312
redwave = siiiwav*(data[rnum]['HA6562_VEL']+2388)/(299792) + siiiwav
low = redwave - 7.5
high = redwave + 45 #lam[np.where(log_spec[rnum] == 0)[0][0]-1]
waverange = lam[np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]
fluxrange = log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]
fluxave = np.mean(fluxrange)
#w0 = waverange[np.where(np.max(log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]) == log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]])]
#w0 = float(w0)
fluxmax = np.max(log_spec[rnum][np.where(lam > low)[0][0]:np.where(lam > high)[0][0]]) - fluxave
dopv = 126.47 - 0.00978*siiiwav
delwave = (siiiwav * dopv * 10**13) / (3 * 10**8 * 10**10)

p0list = np.array([fluxave, fluxmax, redwave, delwave])

siiiwav*(data[rnum]['HA6562_VEL']+2388)/(299792)

In [ ]:
def gaussian(x, C, a, w0, sig):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * sig ** 2)) + C
    
def gaussian_noC(x, a, w0, sig):
    return a * np.exp((-(x-w0) ** 2)/ (2 * sig ** 2))

In [ ]:
param, paramcov = curve_fit(gaussian, waverange, fluxrange, p0list)
flux = quad(gaussian_noC, redwave - 10 * delwave, redwave + 10 * delwave, args=(param[1], param[2], param[3]))[0]
signal = param[1]
noise1 = np.std(log_spec[rnum][np.where(lam > redwave - 175)[0][0]:np.where(lam > redwave - 75)[0][0]])
noise2 = np.std(log_spec[rnum][np.where(lam > redwave + 75)[0][0]:np.where(lam > redwave + 175)[0][0]])
noise = (noise1 + noise2)/2
print(f'For region {rnum} a [SIII]6312 flux value of {flux} was found with a S/N of {signal/noise}')

In [ ]:
param[1]

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 4)

plt.axvline(siiiwav, color='green', linestyle='--', label=f'{siiiwav} Angstrom')
plt.axvline(param[2], color='cyan', linestyle='--', label='Redshifted [S III]6312')
plt.axvline(6300*(data[rnum]['HA6562_VEL']+2388)/(299792) + 6300, color='blue', linestyle='--', label='Redshifted [O I]6300')
plt.axvline(6363*(data[rnum]['HA6562_VEL']+2388)/(299792) + 6363, color='pink', linestyle='--', label='Redshifted [O I]6363')
plt.plot(lam, log_spec[rnum], color = 'red', label='Emission Spectrum')
plt.fill_between(lam[np.where(lam > redwave + 75)[0][0]:np.where(lam > redwave + 175)[0][0]], param[0]*100, color='purple', alpha = 0.15, label='Noise')
plt.fill_between(lam[np.where(lam > redwave - 175)[0][0]:np.where(lam > redwave - 75)[0][0]], param[0]*100, color='purple', alpha = 0.15)
plt.plot(waverange, gaussian(waverange, param[0], param[1], param[2], param[3]), color = 'black', label='Gaussian Fit')

plt.xlim(low - 200, redwave + 200)
plt.ylim(0, param[0] + 1.5*param[0])
plt.legend(loc='best')
plt.xlabel('Wavelength [Angstrom]', fontsize=15)
plt.ylabel(r'Flux [$\frac{erg}{cm^{2} \cdot s \cdot Angstrom}$]', fontsize=15)

In [ ]:
def refit6312(inwave, influx, phdata, wavelength, galvel, snval):
    
    def gaussian(x, C, a, w0):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a, w0):
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2))

    signoise = np.zeros([len(influx)])
    dopv = 126.47 - 0.00978*wavelength
    delwave = (wavelength * dopv * 10**13) / (3 * 10**8 * 10**10)
    outflux = np.zeros(len(influx))
    for i in range(len(influx)):
        redwave = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = redwave - 7.5
        highb = redwave + 45
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500, redwave])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
        
        fluxmax = np.max(fluxes)
        flux = quad(gaussian_noC, redwave - 10 * delwave, highb, args=(fluxmax-param[0], param[2]))[0]
        signal = fluxmax-param[0]
        noise1 = np.std(influx[i][np.where(inwave > redwave - 175)[0][0]:np.where(inwave > redwave - 75)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > redwave + 75)[0][0]:np.where(inwave > redwave + 175)[0][0]])
        noise = (noise1 + noise2)/2
        fluxpeak = np.max(influx[i][np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]])
        rangepeak = np.max(fluxes)
        signoise[i] = signal/noise
        
        if signal/noise > snval and flux > 0: #and fluxpeak == rangepeak: 
            outflux[i] = flux
        else: 
            outflux[i] = np.nan
    
    return outflux, signoise

In [ ]:
ref6312, sn6312 = refit6312(lam, log_spec, data, 6312, 2388, 3)
ref6312

In [ ]:
plt.scatter(ref6312, siiidata['SIII6312_FLUX_REFIT'], s=10, c='red', alpha = 0.5, label='[SIII]6312')
plt.plot(np.linspace(-50, 10**5, 1000), np.linspace(-50, 10**5, 1000), c='black', label='1-to-1 line')
plt.xlabel(r'Eric - Refitted Lines NII [$\frac{erg}{cm^{2}s}$]')
plt.ylabel(r'K. Kreckel - Refitted Lines NII [$\frac{erg}{cm^{2}s}$]')
plt.legend(loc='upper left')
plt.xlim(0, 0.2 * 10**5)
plt.ylim(0, 0.2 * 10**5)
plt.grid()

In [ ]:
count = 0
for i in range(len(siiidata['SIII6312_FLUX_REFIT'])):
    if siiidata[i]['SIII6312_FLUX_REFIT'] > 3 * siiidata[i]['SIII6312_FLUX_REFIT_ERR']:
        count += 1
print(count)

In [ ]:
len(np.where(np.isnan(ref6312)==False)[0])

In [ ]:
count = 0
regs = np.array([])
for i in range(len(siiidata['SIII6312_FLUX_REFIT'])):
    if siiidata[i]['SIII6312_FLUX_REFIT'] > 3 * siiidata[i]['SIII6312_FLUX_REFIT_ERR'] and np.isnan(ref6312[i])==True:
        count += 1
        regs = np.append(regs,int(i))
print(count)
regs

# Correct refits for extinction

In [ ]:
data.add_columns([ref5755, ref6312, ref7319, ref7330], names=('NII5754_FLUX_REFIT','SIII6312_FLUX_REFIT','OII7319_FLUX_REFIT', 'OII7330_FLUX_REFIT'))

In [ ]:
def corr(indata):
    list7319 = [indata[i]['OII7319_FLUX_CORR']/indata[i]['OII7319_FLUX'] for i in range(len(indata))]
    new7319 = [indata[i]['OII7319_FLUX_REFIT'] * list7319[i] for i in range(len(indata))]
    
    list7330 = [indata[i]['OII7330_FLUX_CORR']/indata[i]['OII7330_FLUX'] for i in range(len(indata))]
    new7330 = [indata[i]['OII7330_FLUX_REFIT'] * list7330[i] for i in range(len(indata))]
    
    list5755 = [indata[i]['NII5754_FLUX_CORR']/indata[i]['NII5754_FLUX'] for i in range(len(indata))]
    new5755 = [indata[i]['NII5754_FLUX_REFIT'] * list5755[i] for i in range(len(indata))]
    
    list6312 = [indata[i]['SIII6312_FLUX_CORR']/indata[i]['SIII6312_FLUX'] for i in range(len(indata))]
    new6312 = [indata[i]['SIII6312_FLUX_REFIT'] * list6312[i] for i in range(len(indata))]
    
    indata.add_columns([new5755, new6312, new7319, new7330], names=('NII5754_FLUX_CORR_REFIT', 'SIII6312_FLUX_CORR_REFIT', 'OII7319_FLUX_CORR_REFIT', 'OII7330_FLUX_CORR_REFIT'))

    return indata

In [ ]:
refitdata = corr(data)

# Save all refits to computer

In [ ]:
refitdata.write('NGC4254_refitdata.fits', overwrite=True)  #, overwrite=True